In [1]:
from ase.io import write, read, espresso
from ase import Atom, Atoms
from ase.visualize import view
import numpy as np
import string, sys, math

In [2]:
# Read POSCAR that contains the supercell
inp = read('POSCAR') #lattice constant information
orig = Atoms(inp)
chemical = inp.get_chemical_symbols()
atoms_position = inp.get_scaled_positions()
print(inp.cell)

%mkdir str

Cell([12.253159523, 10.6115484238, 17.1177005768])


| Defect type | Centered atoms | Considered charge states | Recipe 
|-|-|-|-|
| polaron | Rh | -1 | 5% elongation
| polaron | Cu | -1 | 5% elongation
| V_Cu | V_Cu | Neutral, -1, -2 | 5% contraction
| Cu@Fe | Cu@Fe | Neutral, -1, -2 | no bonding change

In [3]:
#low accuracy input options
data = dict({
         'restart_mode':       'from_scratch',
         'calculation':        'relax',
         'verbosity':          'low',
         'nstep':              400,
         'tprnfor':            True,
         'etot_conv_thr':      1e-3, #default: 1e-4 
         'forc_conv_thr':      1e-2, #default: 1e-3 
         'wf_collect':         True,
         'disk_io':            'low',
         'tot_charge':         0.0,
         'ecutwfc':            80.0,
         'ecutrho':            320.0,
         'nosym':              True,
         'occupations':        'smearing',
         'smearing':           'mv',
         'input_dft':          'scan',
         'esm_bc':             'pbc',
         'electron_maxstep':   800,
         'conv_thr':           1e-4, #default: 1e-6
         'mixing_mode':        'plain',
         'mixing_beta':        0.7,
         'nspin':              1,
         'diagonalization':    'david',
         'startingwfc':        'atomic+random',
         'ion_dynamics':       'bfgs',
         'pot_extrapolation':  'atomic',
         'wfc_extrapolation':  'none',    
#        'pseudo_dir':         '/home/552/thl552/raijin_home_2019-11-15/thl552/workspace/collins/qe/pp/', #raijin
         'pseudo_dir':         '/home01/x1837a02/program/pp_oncv/', #nurion
        })

pseudo = dict({
    'O': 'O_ONCV_PBE-1.2.upf',
    'Rh': 'Rh_ONCV_PBE-1.2.upf',
    'Cu': 'Cu_ONCV_PBE-1.2.upf',
    'H': 'H_ONCV_PBE-1.2.upf'
})

kspacing = None #gamma point only, orignal value: 0.05

In [4]:
####polaron @Rh

#Scale factor sf - denotes how much distortion from the original bond is needed (in %).
sf = 1.05 #input

##To get bonding information of center atom and surrounding atoms
#atomic number of center atoms from vesta
vesta_center_atom = 6 #input
center_atom = vesta_center_atom - 1

#atomic number of sorrounding atoms of the center atom from vetsta
vesta_surrounding_atoms = np.array([131, 133, 142, 180, 182, 189]) #input
atoms = vesta_surrounding_atoms - 1

#To get bonding distances
distance = []
for i in atoms:
    distance.append(inp.get_distance(center_atom, i, mic=True))
distance = np.array(distance)

#Adjust the bonds using scale factor
scaled_distance = distance * sf

#Set new surrounding atomic position with periodic boundary conditions
for i, j in zip(atoms, scaled_distance):
    inp.set_distance(center_atom, i, j, fix=0, mic=True)

#output
print("--------")
print("center atom:",chemical[center_atom], vesta_center_atom)
print("--------")
print("[orig]")
for i, j, k in zip(atoms, distance, vesta_surrounding_atoms):
    print(chemical[i], "|   atomic number: ",k,"|   bond length: ", format(j,".3f"))
print("--------")
print("[new]")
for i, j, k in zip(atoms, scaled_distance, vesta_surrounding_atoms):
    print(chemical[i], "|   atomic number: ",k,"|   bond length: ", format(j,".3f"))

path = %pwd
%cd $path/str
pw_in = open('%.2f_polaron_rh.in'%sf, 'w')
espresso.write_espresso_in(pw_in, inp, input_data = data, pseudopotentials = pseudo, kspacing = kspacing, crystal_coordinates = True, koffset=(0, 0, 0))
pw_in.close()
write('%.2f_polaron_rh.vasp'%sf, inp, vasp5 = True)
%cd ..

--------
center atom: Rh 6
--------
[orig]
O |   atomic number:  131 |   bond length:  2.038
O |   atomic number:  133 |   bond length:  2.038
O |   atomic number:  142 |   bond length:  2.038
O |   atomic number:  180 |   bond length:  2.038
O |   atomic number:  182 |   bond length:  2.038
O |   atomic number:  189 |   bond length:  2.038
--------
[new]
O |   atomic number:  131 |   bond length:  2.140
O |   atomic number:  133 |   bond length:  2.140
O |   atomic number:  142 |   bond length:  2.140
O |   atomic number:  180 |   bond length:  2.140
O |   atomic number:  182 |   bond length:  2.140
O |   atomic number:  189 |   bond length:  2.140
/home/taehun/project_curho2/defects/str_model/str
/home/taehun/project_curho2/defects/str_model


In [5]:
####polaron @cu
inp = read('POSCAR')

#Scale factor sf - denotes how much distortion from the original bond is needed (in %).
sf = 1.05 #input

##To get bonding information of center atom and surrounding atoms
#atomic number of center atoms from vesta
vesta_center_atom = 78 #input
center_atom = vesta_center_atom - 1

#atomic number of sorrounding atoms of the center atom from vetsta
vesta_surrounding_atoms = np.array([142, 158]) #input
atoms = vesta_surrounding_atoms - 1

#To get bonding distances
distance = []
for i in atoms:
    distance.append(inp.get_distance(center_atom, i, mic=True))
distance = np.array(distance)

#Adjust the bonds using scale factor
scaled_distance = distance * sf

#Set new surrounding atomic position with periodic boundary conditions
for i, j in zip(atoms, scaled_distance):
    inp.set_distance(center_atom, i, j, fix=0, mic=True)

#output
print("--------")
print("center atom:",chemical[center_atom], vesta_center_atom)
print("--------")
print("[orig]")
for i, j, k in zip(atoms, distance, vesta_surrounding_atoms):
    print(chemical[i], "|   atomic number: ",k,"|   bond length: ", format(j,".3f"))
print("--------")
print("[new]")
for i, j, k in zip(atoms, scaled_distance, vesta_surrounding_atoms):
    print(chemical[i], "|   atomic number: ",k,"|   bond length: ", format(j,".3f"))

path = %pwd
%cd $path/str
pw_in = open('%.2f_polaron_cu.in'%sf, 'w')
espresso.write_espresso_in(pw_in, inp, input_data = data, pseudopotentials = pseudo, kspacing = kspacing, crystal_coordinates = True, koffset=(0, 0, 0))
pw_in.close()
write('%.2f_polaron_cu.vasp'%sf, inp, vasp5 = True)
%cd ..

--------
center atom: Cu 78
--------
[orig]
O |   atomic number:  142 |   bond length:  1.812
O |   atomic number:  158 |   bond length:  1.844
--------
[new]
O |   atomic number:  142 |   bond length:  1.902
O |   atomic number:  158 |   bond length:  1.936
/home/taehun/project_curho2/defects/str_model/str
/home/taehun/project_curho2/defects/str_model


In [6]:
####vacancy cu
inp = read('POSCAR')

##To get bonding information of center atom and surrounding atoms
#atomic number of center atoms from vesta
vesta_vacancy_atom = 78 #input
vacancy_atom = vesta_vacancy_atom - 1

#atomic number of sorrounding atoms of the center atom from vetsta
vesta_surrounding_atoms = np.array([142, 158]) #input
atoms = vesta_surrounding_atoms - 1

#To get bonding distances
distance = []
for i in atoms:
    distance.append(inp.get_distance(vacancy_atom, i, mic=True))
distance = np.array(distance)

#output
print("--------")
print("vacancy atom:",chemical[vacancy_atom], vesta_vacancy_atom)
print("--------")

#creation of vacancy    
del inp[vacancy_atom]

path = %pwd
%cd $path/str
pw_in = open('%.2f_vacancy_cu.in'%sf, 'w')
espresso.write_espresso_in(pw_in, inp, input_data = data, pseudopotentials = pseudo, kspacing = kspacing, crystal_coordinates = True, koffset=(0, 0, 0))
pw_in.close()
write('%.2f_vacancy_cu.vasp'%sf, inp, vasp5 = True)
%cd ..    

--------
vacancy atom: Cu 78
--------
/home/taehun/project_curho2/defects/str_model/str
/home/taehun/project_curho2/defects/str_model


In [7]:
####Antisite cu@Rh (no structural distortion)
inp = read('POSCAR')

#atomic number of center atoms from vesta
vesta_center_atom = 6 #input
center_atom = vesta_center_atom - 1
chemical = inp.get_chemical_symbols()
chemical[center_atom] = 'Cu'
inp.set_chemical_symbols(chemical)

path = %pwd
%cd $path/str
pw_in = open('1.00_cu_at_fe.in', 'w')
espresso.write_espresso_in(pw_in, inp, input_data = data, pseudopotentials = pseudo, kspacing = kspacing, crystal_coordinates = True, koffset=(0, 0, 0))
pw_in.close()
write('1.00_cu_at_fe.vasp', inp, vasp5 = True)
%cd ..    

#output
print("--------")
print("Antistie atom:",chemical[center_atom], vesta_center_atom)
print("--------")

/home/taehun/project_curho2/defects/str_model/str
/home/taehun/project_curho2/defects/str_model
--------
Antistie atom: Cu 6
--------
